In [2]:
from keras.layers import Input, Dense, Dropout, concatenate, Flatten, BatchNormalization,LeakyReLU,add
from keras.models import Model
from keras.layers.normalization import BatchNormalization
import h5py
import time
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
f = h5py.File('.\Input_Patches_3Dsyn2.mat', 'r')
dataNoisy= f.get('dn_patch')

In [4]:
# Random Permute for the patches for training
# ind = np.random.permutation(len(dataNoisy))
dataNoisy = np.array(dataNoisy)
# dataNoisy = dataNoisy[ind]


INPUT_SIZE1 = dataNoisy.shape[0]
INPUT_SIZE2 = dataNoisy.shape[1]


input_img = Input(shape=(INPUT_SIZE2,))


encoded1 = Dense(128)(input_img)
encoded1 = BatchNormalization()(encoded1)
encoded1 = LeakyReLU(0.2)(encoded1)
encoded1 = Dense(128)(encoded1)
encoded1 = BatchNormalization()(encoded1)
encoded1 = LeakyReLU(0.2,name='encoder1')(encoded1)

skip1 = Dense(128)(encoded1)
skip1 = BatchNormalization()(skip1)
skip1 = LeakyReLU(0.2)(skip1)

encoded2 = Dense(64)(encoded1)
encoded2 = BatchNormalization()(encoded2)
encoded2 = LeakyReLU(0.2)(encoded2)
encoded2 = Dense(64)(encoded2)
encoded2 = BatchNormalization()(encoded2)
encoded2 = LeakyReLU(0.2,name='encoder2')(encoded2)

skip2 = Dense(64)(encoded2)
skip2 = BatchNormalization()(skip2)
skip2 = LeakyReLU(0.2)(skip2)

encoded3 = Dense(32)(encoded2)
encoded3 = BatchNormalization()(encoded3)
encoded3 = LeakyReLU(0.2)(encoded3)
encoded3 = Dense(32)(encoded3)
encoded3 = BatchNormalization()(encoded3)
encoded3 = LeakyReLU(0.2)(encoded3)

skip3 = Dense(32)(encoded3)
skip3 = BatchNormalization()(skip3)
skip3 = LeakyReLU(0.2)(skip3)

encoded4 = Dense(16)(encoded3)
encoded4 = BatchNormalization()(encoded4)
encoded4 = LeakyReLU(0.2)(encoded4)
encoded4 = Dense(16)(encoded4)
encoded4 = BatchNormalization()(encoded4)
encoded4 = LeakyReLU(0.2)(encoded4)

skip4 = Dense(16)(encoded4)
skip4 = BatchNormalization()(skip4)
skip4 = LeakyReLU(0.2)(skip4)

encoded5 = Dense(8)(encoded4)
encoded5 = BatchNormalization()(encoded5)
encoded5 = LeakyReLU(0.2)(encoded5)
encoded5 = Dense(8)(encoded5)
encoded5 = BatchNormalization()(encoded5)
encoded5 = LeakyReLU(0.2)(encoded5)

skip5 = Dense(8)(encoded5)
skip5 = BatchNormalization()(skip5)
skip5 = LeakyReLU(0.2)(skip5)

encoded6 = Dense(4)(encoded5)
encoded6 = BatchNormalization()(encoded6)
encoded6 = LeakyReLU(0.2)(encoded6)
encoded6 = Dense(4)(encoded6)
encoded6 = BatchNormalization()(encoded6)
encoded6 = LeakyReLU(0.2)(encoded6)

skip6 = Dense(4)(encoded6)
skip6 = BatchNormalization()(skip6)
skip6 = LeakyReLU(0.2)(skip6)
# Decoder

decoded6 = Dense(4)(skip6)
decoded6 = BatchNormalization()(decoded6)
decoded6 = LeakyReLU(0.2)(decoded6)
decoded6 = Dense(4)(decoded6)
decoded6 = BatchNormalization()(decoded6)
decoded6 = LeakyReLU(0.2)(decoded6)
decoded6 = concatenate([decoded6, skip6])


decoded5 = Dense(8)(decoded6)
decoded5 = BatchNormalization()(decoded5)
decoded5 = LeakyReLU(0.2)(decoded5)
decoded5 = Dense(8)(decoded5)
decoded5 = BatchNormalization()(decoded5)
decoded5 = LeakyReLU(0.2)(decoded5)
decoded5 = concatenate([decoded5, skip5])

decoded4 = Dense(16)(decoded5)
decoded4 = BatchNormalization()(decoded4)
decoded4 = LeakyReLU(0.2)(decoded4)
decoded4 = Dense(16)(decoded4)
decoded4 = BatchNormalization()(decoded4)
decoded4 = LeakyReLU(0.2)(decoded4)
decoded4 = concatenate([decoded4, skip4])

decoded3 = Dense(32)(decoded4)
decoded3 = BatchNormalization()(decoded3)
decoded3 = LeakyReLU(0.2)(decoded3)
decoded3 = Dense(32)(decoded3)
decoded3 = BatchNormalization()(decoded3)
decoded3 = LeakyReLU(0.2)(decoded3)
decoded3 = concatenate([decoded3, skip3])

decoded2 = Dense(64)(decoded3)
decoded2 = BatchNormalization()(decoded2)
decoded2 = LeakyReLU(0.2)(decoded2)
decoded2 = Dense(64)(decoded2)
decoded2 = BatchNormalization()(decoded2)
decoded2 = LeakyReLU(0.2)(decoded2)
decoded2 = concatenate([decoded2, skip2])

decoded1 = Dense(128)(decoded2)
decoded1 = BatchNormalization()(decoded1)
decoded1 = LeakyReLU(0.2)(decoded1)
decoded1 = Dense(128)(decoded1)
decoded1 = BatchNormalization()(decoded1)
decoded1 = LeakyReLU(0.2)(decoded1)
decoded1 = concatenate([decoded1, skip1])

# decoded = Flatten()(decoded3)
decoded = Dense(INPUT_SIZE2, activation='linear')(decoded1)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3375)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          432128      input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128)          512         dense_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 128)          0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [6]:
# Addig stopping condition
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

batch = 128

# tic()
start=time.time()
history = autoencoder.fit(dataNoisy, dataNoisy, verbose=2,epochs=100, batch_size=batch, shuffle="batch", callbacks=[es, mc],
                          validation_split=0.2)
end = time.time()
running_time = end - start
print('time cost : %.5f sec' % running_time)

Train on 29030 samples, validate on 7258 samples
Epoch 1/100
 - 16s - loss: 0.0463 - val_loss: 0.0454
Epoch 2/100
 - 17s - loss: 0.0448 - val_loss: 0.0442
Epoch 3/100
 - 15s - loss: 0.0440 - val_loss: 0.0436
Epoch 4/100
 - 15s - loss: 0.0434 - val_loss: 0.0432
Epoch 5/100
 - 15s - loss: 0.0430 - val_loss: 0.0429
Epoch 6/100
 - 15s - loss: 0.0428 - val_loss: 0.0427
Epoch 7/100
 - 15s - loss: 0.0425 - val_loss: 0.0425
Epoch 8/100
 - 15s - loss: 0.0423 - val_loss: 0.0424
Epoch 9/100
 - 16s - loss: 0.0422 - val_loss: 0.0422
Epoch 10/100
 - 15s - loss: 0.0420 - val_loss: 0.0421
Epoch 11/100
 - 15s - loss: 0.0419 - val_loss: 0.0421
Epoch 12/100
 - 15s - loss: 0.0418 - val_loss: 0.0420
Epoch 13/100
 - 15s - loss: 0.0417 - val_loss: 0.0419
Epoch 14/100
 - 15s - loss: 0.0416 - val_loss: 0.0419
Epoch 15/100
 - 15s - loss: 0.0415 - val_loss: 0.0418
Epoch 16/100
 - 15s - loss: 0.0415 - val_loss: 0.0418
Epoch 17/100
 - 15s - loss: 0.0414 - val_loss: 0.0418
Epoch 18/100
 - 15s - loss: 0.0413 - val_l

In [8]:
from keras.models import load_model
import scipy.io

model = load_model('best_model.h5')
f = h5py.File('.\Input_Patches_3Dsyn2.mat', 'r')
dataNoise= f.get('dn_patch')

out = model.predict(dataNoise)
scipy.io.savemat('.\Output_Patches_3Dsyn2.mat', mdict={'out': out})